## Applications of Graph Neural Networks

#### Over-Smoothing Problem
The issue of stacking too many GNN layers is that GNNs suffer from the over-smoothing problem. This is essentially that all the node embeddings converge to the same value. This is really bad as we want to use node embeddings to differentiate nodes. 

The **receptive field** is the set of nodes that determine the embedding of a node of interest. In a K-layer GNN, each node has a receptive field of K-hop neighborhood. The shared neighbors quickly grow when we increase the number of hops (number of GNN layers). 

Over-smoothing can be explained via the notion of the receptive field. We know that the embedding of a node is determined by its receptive field. If two nodes have highly-overlapped receptive fields, then their embeddings are highly similar. 

Stacking many GNN layers will lead to nodes having highly-overlapped receptive fields. Node embeddings will be highly similar and suffer from the over-smoothing problem. How do we overcome this?

The first lesson is that we need to be cautious when adding GNN layers. Adding more GNN layers do not always help. 

We can also make GNNs more expressive for when we use shallow GNNs. We can make aggregation/transformation become a deep neural network. We could add layers that do not pass messages. A GNN does not necessarily only contain GNN layers. For example, we could add MLP layers (applied to each node) before and after GNN layers as pre-process layers and post-process layers. **Pre-processing layers**: are important when encoding node features is necessary (eg when nodes represent images/text). **Post-processing layers** are important when reasoning/transformation over node embeddings is needed (eg. graph classification, knowledge graphs). These layers work really well in pratice.

If we absolutely require many layers, we can also add skip connections. The basic idea of skip connections is that before adding shortcuts, the function is F(x) and after adding shortcuts, it becomes F(x) + x. We want to create a mixture of models. N skip connections leads to $2^{N}$ possible paths. Each path could have up to N modules and we automatically get a mixture of shallow GNNs and deep GNNs.

#### Graph Augmentation for GNNs
Its highly unlikely that the raw input graph happens to be the optimal computational graph for embeddings. We may not want to use raw input graphs as computational graphs since input graphs may lack features, graphs may be too sparse, graphs may be too dense or may even be too large.

Graph feature augmentation is when the graph lacks features. Graph structure augementation is done when the graph is too sparse, dense or large.

The standard approaches to feature augmentation include assigning constant values to nodes, and assigning unique IDs to nodes. Assigning unique IDs to nodes uses IDs that can be converted into one-hot vectors.

Feature augmentation is also important when we have some structures that may be really hard to learn by GNNs. 

When we want to augment sparse graphs, we can always add virtual edges and virutal nodes. 

#### GNN Predictions
There are a couple of different prediction heads possible including node level tasks, edge level tasks and graph level tasks. Different task levels require different prediction heads. 

For node level prediction, we can directly make prediction using node embeddings. For edge-level predictions, we can make prediction using pairs of node embeddings. For graph level predictions, we can make prediction using all the node embeddings in our graph.

For supervised learning, the labels are coming from the graphs. For unsupervised learning, there are signals instead that come from the graphs. The differences between the two can be blurry as we still have "supervision" in unsupervised learning. Supervised lables come from specific use cases that include node labels, edge labels, and graph labels. It is best to reduce your task to node/edge/graph labels since they're easiest to work with.

When we  want to compute final loss, we could use two common ones which are classification loss and regression loss. Classification is used for labels with discrete values to predict. Regression is used for labels with continuous values. GNNs can be applied to both settings. The differences between the two are loss function and evaluation metrics.


#### Classification Loss
Cross entropy (CE) is a very common loss function in classification. K-way prediction for i-th data point:

$CE(y^{(i)},\hat{y}^{(i)}=-\sum^{K}_{j=1}y_j^{(i)}log(\hat{y}_{j}^{(i)})$

where:

$y^{(i)} in \mathbb{R}^{K}$ = one-hot label encoding

$\hat{y}^{(i)} in \mathbb{R}^{K}$ = prediction after Softmax

Total loss over all N training examples

$Loss = \sum^{N}_{i=1}CE(y^{(i)},\hat{y}^{(i)})$

#### Regression Loss
For regression tasks we often use mean squared error (MSE) or L2 loss.

K-way regression for data point (i):

$MSE(y^{(i)},\hat{y}^{(i)}) = \sum^{K}_{j=1} (y^{(i)}_{j} - \hat{y}^{(i)}_{j})^2$

where:

$y^{(i)} in \mathbb{R}^{K}$ = Real valued vector of target

$\hat{y}^{(i)} in \mathbb{R}^{K}$ = Real valued vector of predictions

Total loss over all N training examples:

$Loss = \sum^{N}_{i=1}MSE(y^{(i)},\hat{y}^{(i)})$


#### GAN Evaluation Metrics

How do we measure the success of a GNN? We can use accuracy and ROC AUC. For evaluating regression tasks on graphs, we'd typically use root mean square error (RMSE) or mean absolute error (MAE). For classification evaluation metrics, we'd use multi-class classification (which allows us to simply report the accuracy) or binary classification (metrics sensitive to classification threshold which include accuracy, and precision/recall). Metric agnostic to classification threshold is ROC AUC.

#### Train / Validation / Test Sets

Splitting the dataset is an important part of deep learning. Fixed split is when we split our dataset once. A concern is that sometimes we cannot gurantee that the test set will really be held out. Random split is when we will randomly split our dataset into training/validation/test. 

Splitting a graph is really different from splitting other datasets. For node classification, each data point is a node. Here, data points are not independent.

Solution 1 (transductive setting) for splitting graphs is that the input graph can be observed in all the dataset splits (training, validation and test set). We will only split the node labels. At training time, we compute embeddings using the entire graph and train using only specific node's labels. At validation time, we compute embeddings using the entire grpah and evaluate on only specific node labels.

Solution 2 (inductive setting) is when we break the edges between splits to get multiple graphs. This gives us indepedent graphs that are independent and predictions of one node will not affect prediction on another node. 


Transductive setting: all sets are on the same graph and the dataset consists of one graph. The entire graph can be ibserved in all dataset splits, we only split the lables. Only applicable to node/edge prediction tasks. 

Inductive setting: All sets are on different graphs. The dataset consists of multiple graphs. Each split can only observe the graphs within the split. A successful model should generalize to unseen graphs and this is applicable to node/edge/graph tasks. 

#### Node Classification
For transductive node classification, all the splits can observe the entire graph structure, but can only observe the labels of their respective nodes. For inductive node classification, we would have a dataset of 3 graphs and each split contains an independent graph. 

#### Graph Classification
Only the inductive setting is well defined for graph classification. This is because we have to test on unseen graphs. Suppose we have a dataset of 5 graphs, each split will contain indepdenet graphs. 

#### Link Prediction
The goal is to predict missing edges and setting up link prediction is tricky. It is an unsupervised/self-supervised task. We need to create labels and dataset splits on our own. Concretely, we'll need to hide some edges from the GNN and let the GNN predict if the edges exist.

For link prediction, we'll split up the edges twice. The first step is to assign 2 types of edges in the original graph. We'll call one message edges and the other **supervision edges**. The message edges are used for GNN message passing and the supervision edges are used for computing objectives. After step 1, only message edges will remain in the graph. Supervision edges are used as supervision for edge predictions made by the model and will not be fed into the GNN. 

Step 2 consists of splitting edges into train/validation/test. From there we have the option of inductive link prediction split which gives us multiple independent graphs to use. In the train or val or test set, each graph will have 2 types of edges (message and supervision). 

The other option is transductive link prediction split. This it the default setting when people talk about link prediction. The entire graph can be observed in all dataset splits. But since edges are both part of graph strucutres and the supervision, we need to hold out validation/test edges. To train the training set further, we need to hold out supervision edges for the training set.

## Theory of Graph Neural Networks

#### A Couple of General Tips
Data preprocessing is highly important! Node attributes can vary a lot so it's important to use normalization. A really good optimizer is ADAM as it's relatively robust to learning rate. ReLU activation functions tend to work well but some alternatives include LeakyReLU, SWISH, and rational activation. There's no activation function at your output layer. Ensure that bias terms are added in each layer.

#### How Powerful are GNNs?
Many GNN models have been proposed and they have different expressive powers. 

In each layer, a GNN aggregates neighboring node embeddings. A GNN generates node embeddings through a computational graph defined by the neighborhood. GNN only sees node features (not IDs). A GNN will generate the same embedding if the computational graphs are the same and node features are identical. In general different local neighborhoods define different computational graphs. 

Computational graphs are identical to rooted subtree structures around each node. Rooted subtree structures are defined recursively unfolding neighboring nodes from the root nodes. GNN's node embeddings capture rooted subtree structures. Most expressive GNN maps different rooted subtree into different node embeddings. Most expressive GNN should map subtrees to the node embeddings injectively. 


A key observation is that subtrees of the same depth can be recursively characterized from the leaf nodes to the root nodes. If each step of the GNN's aggregation can fully retain the neighboring information, the generated node embeddings can distinguish different rooted subtrees. In other words, most expressive GNN would use an injective neighbor aggregation function at each step. Maps different neighbors to different embeddings.


A really important part of the expressive power of GNNs is the neighbor aggregation function. 

